In [4]:
import os
import geopandas as gpd
import folium
from sqlalchemy import create_engine

# 1) DB 연결 & GeoDataFrame 로드
db_url = os.getenv("DB_DSN")
engine = create_engine(db_url)
gdf = gpd.read_postgis(
    "SELECT grid_id, year, area, geom FROM bld_in_grid",
    con=engine,
    geom_col="geom",
    crs="EPSG:5179"
)
# 2) WGS84 위경도로 변환 (Folium이 WGS84 기준)
gdf_wgs = gdf.to_crs(epsg=4326)

# 3) Folium 지도 생성 (서울시청 중심)
m = folium.Map(
    location=[37.5665, 126.9780],
    zoom_start=11,
    tiles="CartoDB.Positron"
)

# 4) GeoJson 레이어 추가 — 여기서는 연도(year)에 따라 색상을 입히는 예시
folium.Choropleth(
    geo_data=gdf_wgs,
    data=gdf_wgs,
    columns=["grid_id", "year"],
    key_on="feature.properties.grid_id",
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="건물 준공연도"
).add_to(m)

# 5) 팝업/툴팁 추가 (격자 클릭 시 상세 정보)
folium.GeoJson(
    gdf_wgs,
    style_function=lambda feat: {
        "color": "gray",
        "weight": 0.5,
        "fillOpacity": 0
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["grid_id", "year", "area"],
        aliases=["Grid ID", "준공연도", "면적(㎡)"],
        localize=True
    )
).add_to(m)

# 6) 레이어 컨트롤
folium.LayerControl().add_to(m)

# 7) 지도 출력
m

ArgumentError: Expected string or URL object, got None